In [ ]:
from pyspark.sql.types import IntegerType, StringType, StructType, FloatType
from pyspark.sql.functions import *

In [ ]:
#variables globales a usar en el programa
CSV_DATA_PATH = "dbfs:/FileStore/tables/walmart_data.csv"
PARTITIONS_NUMBER = 30 #numero de archivos a partir el csv de walmart 
RAW_PATH = "dbfs:/FileStore/raw/"
SILVER_PATH = "dbfs:/FileStore/silver/"
SILVER_CHECKPOINT_LOCATION = "dbfs:/FileStore/silver/check"

In [ ]:
schema = StructType()\
         .add("store",IntegerType())\
         .add("Date",StringType())\
         .add("weekly_sales",FloatType())\
         .add("is_holiday",IntegerType())\
         .add("temperature",FloatType())\
         .add("fuel_price",FloatType())\
         .add("cpi",FloatType())\
         .add("unemployment",FloatType())

In [ ]:
#leer data de CSV de walmart sales
df_walmart = spark.read\
                  .format("csv")\
                  .option("header",True)\
                  .schema(schema)\
                  .load(CSV_DATA_PATH)

In [ ]:
#partir el conjunto de datos de walmart en n partes
partition_data = df_walmart.repartition(PARTITIONS_NUMBER)

In [ ]:
#guardando las n particiones del conjunto de datos en la capa raw
partition_data.write\
              .format("parquet")\
              .option("header",True)\
              .save(RAW_PATH)

In [ ]:
#simulo el proceso en streaming que va tomando un archivo con cada ejecución
stream = spark.readStream\
              .format("parquet")\
              .schema(schema)\
              .option("header",True)\
              .option("ignoreLeadingWhiteSpace",True)\
              .option("mode","dropMalFormed")\
              .option("maxFilesPerTrigger",1)\
              .load(RAW_PATH)

In [ ]:
#escribo en la capa silver los datos tomados por medio del streaming
stream.writeStream\
      .option("checkPointLocation",SILVER_CHECKPOINT_LOCATION)\
      .format("parquet")\
      .outputMode("append")\
      .queryName("silver_stream")\
      .start(SILVER_PATH)

In [ ]:
#para detener los procesos de streaming
for s in spark.streams.active:
      s.stop()